In [1]:
import pandas as pd
import sys
sys.path.append('../src')
from scrapy import Scrapy
from convert_json_csv import ConvertJSONToCsv

In [2]:
app_ids = ['com.combanketh.mobilebanking', 'com.boa.boaMobileBanking', 'com.dashen.dashensuperapp']

In [ ]:

csv = ConvertJSONToCsv()
for app_id in app_ids:
    scrapy = Scrapy()
    data = scrapy.save_data_json('../data/',f'data-{app_id}.json', app_id)
    print(f'{app_id} has {len(data)} reviews')


Data already imported
com.combanketh.mobilebanking has 0 reviews
Data already imported
com.boa.boaMobileBanking has 0 reviews
Data already imported
com.dashen.dashensuperapp has 0 reviews


In [ ]:
import json
for id in app_ids:
    with open(f'../data/data-{id}.json') as f:
        data = json.load(f)
        csv.to_csv(data, f'{id.split('.')[1]}.csv')

In [3]:
import pandas as pd
from preprocess import clean_data, find_columns_with_missing_value
for id in app_ids:
    bank = id.split('.')[1]
    data = pd.read_csv(f'../data/{bank}.csv')
    print(id.split('.')[1])
    print('='*30)
    cleaned_data = clean_data(data)
    cleaned_data['bank'] = bank
    cleaned_data['source'] = 'play-store'
    print(cleaned_data.head())

    #cleaned_data = clean_data(data)

combanketh
columns above the threshold
                               reviewId       userName  \
0  aa72fd94-aca8-48b2-bae2-29ad7633c7fb  A Google user   
1  9af95d2a-e42c-45ff-a4c9-5c38765df4a4  A Google user   
2  067d3e52-3586-4e02-b088-0411f25d34fb  A Google user   
3  88a64315-9de3-4a35-ac27-5ff37c2833b6  A Google user   
4  c2cf0ff7-85ac-4e7c-902d-a8c5a8d69e22  A Google user   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                               good      5              0   
1  it was good app but it have some issues like i...      2              0   
2                                        